In [ ]:
import gurobipy as gp

# Q1

### Simple way 

In [ ]:
#gp.setParam("OutputFlag",1)
model=gp.Model()
x1=model.addVar(lb=100,ub=1000)
x2=model.addVar(lb=100,ub=1000)
x3=model.addVar(lb=100,ub=1000)
x4=model.addVar()
x4.lb=100
x4.ub=1000
model.setObjective(16*x1+15*x2+20*x3+22*x4,gp.GRB.MAXIMIZE)
model.addConstr(2*x1+x2+3*x3+4*x4<=3000)
model.addConstr(3*x1+4*x2+5*x3+6*x4<=6000)
model.optimize()
print("problem status is", model.status)
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("x1",x1.X)
print("x1",x2.X)
print("x1",x3.X)
print("x1",x4.X)
print("optimal value",model.objval)

In [ ]:
help(model.addVars)


### Better way-using dictionaries

In [ ]:
#Data
products=["Ray","Zapper","Squire","Soaker"]
resources=["plastic","time"]
profit={"Ray":16
        ,"Zapper":15
        ,"Squire":20,
        "Soaker":22}
resAvail={"plastic":3000,"time":6000}
resUse={("Ray","plastic"):2,("Ray","time"):3,
    ("Zapper","plastic"):1,("Zapper","time"):4,
    ("Squire","plastic"):3,("Squire","time"):5,
    ("Soaker","plastic"):4,("Soaker","time"):6
    }

#Model
model=gp.Model()
pvars=model.addVars(products,lb=100,ub=1000,obj=profit,name="products")
model.ModelSense=gp.GRB.MAXIMIZE
model.addConstrs(gp.quicksum(resUse[p,r]*pvars[p] for p in products)<=resAvail[r] 
                 for r in resources)

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")


### Using for loops

In [ ]:
#Data
products=["Ray","Zapper","Squire","Soaker"]
resources=["plastic","time"]
profit={"Ray":16
        ,"Zapper":15
        ,"Squire":20,
        "Soaker":22}
resAvail={"plastic":3000,"time":6000}
resUse={("Ray","plastic"):2,("Ray","time"):3,
    ("Zapper","plastic"):1,("Zapper","time"):4,
    ("Squire","plastic"):3,("Squire","time"):5,
    ("Soaker","plastic"):4,("Soaker","time"):6
    }

#Model
model=gp.Model()
pvars={}
for p in products:
    pvars[p]=model.addVar(lb=100,ub=1000,obj=profit[p],name=p)
model.ModelSense=gp.GRB.MAXIMIZE
for r in resources:
    lhs=0
    for p in products:
        lhs+=resUse[p,r]*pvars[p] 
    model.addConstr(lhs<=resAvail[r]) 
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")


### Alternative (also good) way-using arrays

A matter of taste which one you prefer. Good to know both as sometimes one may be more convenient than the other. 

In [ ]:
import numpy as np
profit = np.array([16,15, 20, 22])
res_use=np.array([[2,1,3,4],[3,4,5,6]])
res_avail = np.array([3000, 6000])   
n = len(profit)                                    # number of variables
m = len(res_avail)                                 # number of constraints
assert res_use.shape == (m, n)
model=gp.Model("Q1")
pvars=model.addVars(n,name="products",lb=100,ub=1000)
model.setObjective(gp.quicksum(profit[i]*pvars[i] for i in range(n)),gp.GRB.MAXIMIZE)
cons=model.addConstrs(gp.quicksum(res_use[i,j]*pvars[j] for j in range(n))<=res_avail[i]
                     for i in range(m))
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

# Q2

In [ ]:
makeCosts=[6,13,20]
buyACosts=[12,15,21]
buyBCosts=[11,16,23]
laborUse=[1,2,3]
demands=[100,80,70]
aProdLimit=60
laborAvail=200
n=3 #vars num
model=gp.Model()
M=model.addVars(n,name="make",obj=makeCosts)
A=model.addVars(n,name="BuyA",obj=buyACosts)
B=model.addVars(n,name="BuyB",obj=buyBCosts)
#by default minimization
model.addConstrs(M[i]+A[i]+B[i]==demands[i] for i in range(n)) #sattisfy demands
model.addConstr(gp.quicksum(laborUse[i]*M[i] for i in range(n))<=laborAvail)
model.addConstr(gp.quicksum(A[i] for i in range(n))<=aProdLimit)
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

# Q3

In [ ]:
Bonds=["A","B","C","D","E"]
n=len(Bonds)
returns=[.095,.08,.09,.09,.09]
longMaturity=[True,False,True,True,False]
highRisk=[1,0,0,1,1]
taxFree=[1,1,0,1,0]

totalInvestment=100000
shortTermMinRatio=.5
lowRiskRatio=.55
taxFreeMinRatio=.3
taxFreeRetMinRatio=.4

model=gp.Model()
x=model.addVars(n,name=Bonds,obj=returns)
model.ModelSense=gp.GRB.MAXIMIZE
model.addConstr(gp.quicksum(x[i] for i in range(n))==totalInvestment)
model.addConstr(gp.quicksum(x[i] for i in range(n) if not longMaturity[i])
                >=totalInvestment*shortTermMinRatio)
model.addConstr(gp.quicksum(x[i] for i in range(n) if not highRisk[i])
                >=totalInvestment*lowRiskRatio)
model.addConstr(gp.quicksum(x[i] for i in range(n) if taxFree[i])
                >=totalInvestment*taxFreeMinRatio)
model.addConstr(gp.quicksum(returns[i]*x[i] for i in range(n) if taxFree[i])
                >=taxFreeRetMinRatio*gp.quicksum(returns[i]*x[i] for i in range(n)))

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")
#model.write("Q3.lp")
#with open("Q3.lp",'r') as f:
#    print(f.read())

# Q4

In [ ]:
groves=['A','B','C']
plants=['D','E','F']
distances={('A','D'):21,('A','E'):50,('A','F'):40,
           ('B','D'):35,('B','E'):30,('B','F'):22,
           ('C','D'):55,('C','E'):20,('C','F'):25}
costPerMile=1.5
capacities={'D':2000,'E':6000,'F':2250}
supplies={'A':2750,'B':4000,'C':3000}

model=gp.Model()
x=model.addVars(distances,name='flow')
model.setObjective(costPerMile*gp.quicksum(distances[g,p]*x[g,p] 
                                           for g in groves
                                           for p in plants))
model.addConstrs(gp.quicksum(x[g,p] for p in plants)==supplies[g] 
                 for g in groves)
model.addConstrs(gp.quicksum(x[g,p] for g in groves)<=capacities[p] 
                 for p in plants)

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

# Q5

In [ ]:
productionCosts=[250,253,255,253,250,255]
demands=[1000,4500,6000,4500,3500,2000]
maxProduction=[4000,3500,4000,4500,4000,2500]
periods=6
minInventory=1500
maxInventory=6000
startingInventory=1800
inventoryCost=4

model=gp.Model()
P=model.addVars(periods,name="P",obj=productionCosts,
               lb=[0.5*i for i in maxProduction],ub=maxProduction)
I=model.addVars(periods,name="I",obj=inventoryCost,lb=1500,ub=6000)
model.addConstr(I[0]==startingInventory+P[0]-demands[0])
model.addConstrs(I[t]==I[t-1]+P[t]-demands[t] for t in range(1,periods))
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

# Q6

In [ ]:
investments=["A","B","C","D"]
available={"A":[1,2,3,4,5,6],"B":[1,3,5],"C":[1,4],"D":[1]}
maturity={"A":1,"B":2,"C":3,"D":6}
yields={"A":1.018,"B":1.035,"C":1.058,"D":1.11}
yearCommitments=[0,250000,0,250000,0,300000]
model=gp.Model()
myVars={}
for inv in investments:
    myVars[inv]=model.addVars(available[inv],name=inv)
obj=0
for inv in investments:
    obj+=myVars[inv][1]
model.setObjective(obj)

for t in range(1,7):
    lhs=0
    for inv in investments:
        if (t-maturity[inv]+1) in available[inv]: #investments maturing at end of t
            lhs+=yields[inv]*myVars[inv][t-maturity[inv]+1]
    rhs=yearCommitments[t-1] #-1 because counter starts from 0
    for inv in investments:
        if (t+1) in available[inv]:
            rhs+=myVars[inv][t+1]
    model.addConstr(lhs==rhs)

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

#model.write("Q6.lp")
#with open("Q6.lp",'r') as f:
#       print(f.read())
